# Serie A Historical Data Collection

Fetching Serie A match data from 2009 onwards using **football-data.co.uk** (free, reliable, goes back 15+ years!).

**Goal:** Build a database to compare standings at specific matchdays across different seasons.

## Step 1: Install Dependencies

In [ ]:
!pip install pandas sqlalchemy tqdm --break-system-packages

## Step 2: Import Libraries

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from tqdm import tqdm
from datetime import datetime
import os
import time

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✓ Libraries loaded!")

## Step 3: Setup Database Path

In [ ]:
# Create data directory if it doesn't exist
os.makedirs("data", exist_ok=True)
DB_PATH = "data/serie_a.db"

# Create SQLAlchemy engine
engine = create_engine(f"sqlite:///{DB_PATH}")

print(f"Database will be stored at: {DB_PATH}")

## Step 4: Determine Current Season

Let's figure out what season we're in (new season starts in August).

In [ ]:
year = datetime.now().year
month = datetime.now().month

if month >= 8:  # New season starts in August
    current_season = f"{year}-{year+1}"
else:
    current_season = f"{year-1}-{year}"

print(f"Current season: {current_season}")

## Step 5: Build Season URLs

football-data.co.uk has data going back to 2009. Let's build URLs for all seasons.

In [ ]:
# Build season mapping from 2009 to current
SEASONS = {}

for y in range(2009, year + 1):
    season_label = f"{y}-{y+1}"
    suffix = f"{str(y)[-2:]}{str(y+1)[-2:]}"
    url = f"https://www.football-data.co.uk/mmz4281/{suffix}/I1.csv"
    SEASONS[season_label] = url

print(f"Will fetch {len(SEASONS)} seasons:")
print(f"From: {list(SEASONS.keys())[0]}")
print(f"To: {list(SEASONS.keys())[-1]}")

## Step 6: Test with One Season

Before fetching all seasons, let's test with one to see what data we get.

In [ ]:
# Test with 2023-24 season
test_url = SEASONS['2023-2024']
print(f"Testing URL: {test_url}")

test_df = pd.read_csv(test_url)
print(f"\n✓ Successfully fetched {len(test_df)} rows")
print(f"\nColumns available:")
print(test_df.columns.tolist())

In [ ]:
# Look at first few matches
print("Sample data:")
test_df[['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']].head(10)

## Step 7: Fetch All Seasons

Now let's fetch all seasons. **This will take 1-2 minutes** (we add a small delay between requests to be polite).

In [ ]:
all_matches = []

for season, url in tqdm(SEASONS.items(), desc="Fetching Serie A seasons"):
    try:
        df = pd.read_csv(url)
        
        if df.empty:
            print(f"⚠️ {season} returned empty file")
            continue
        
        # Keep only played matches (FTHG = Full Time Home Goals)
        df = df[df["FTHG"].notna() & df["FTAG"].notna()]
        
        # Add season label
        df["season_label"] = season
        
        all_matches.append(df)
        
        # Be nice to the server
        time.sleep(0.3)
        
    except Exception as e:
        print(f"⚠️ Failed to read {season}: {e}")

print(f"\n✓ Successfully fetched {len(all_matches)} seasons!")

## Step 8: Combine All Data

In [ ]:
# Combine all seasons into one DataFrame
matches = pd.concat(all_matches, ignore_index=True)

print(f"Total matches: {len(matches)}")
print(f"Seasons: {matches['season_label'].nunique()}")
print(f"Date range: {matches['Date'].min()} to {matches['Date'].max()}")

## Step 9: Clean and Standardize Data

In [ ]:
# Select relevant columns and rename
cols = ["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "season_label"]
matches = matches[cols].rename(
    columns={
        "Date": "date",
        "HomeTeam": "home_team",
        "AwayTeam": "away_team",
        "FTHG": "home_goals",
        "FTAG": "away_goals",
        "FTR": "result",  # H=Home win, A=Away win, D=Draw
    }
)

print("✓ Columns renamed")
matches.head()

## Step 10: Parse Dates

Different seasons use different date formats, so we'll try both common formats.

In [ ]:
def parse_date(x):
    """Try both DD/MM/YY and DD/MM/YYYY formats"""
    for fmt in ("%d/%m/%y", "%d/%m/%Y"):
        try:
            return pd.to_datetime(x, format=fmt)
        except Exception:
            continue
    return pd.NaT

print("Parsing dates...")
matches["date"] = matches["date"].apply(parse_date)

# Remove any rows where date parsing failed
before = len(matches)
matches = matches.dropna(subset=["date"])
after = len(matches)

print(f"✓ Dates parsed successfully")
print(f"Dropped {before - after} rows with invalid dates")
print(f"\nDate range: {matches['date'].min()} to {matches['date'].max()}")

## Step 11: Check the Data

In [ ]:
# See unique teams
all_teams = set(matches['home_team'].unique()) | set(matches['away_team'].unique())
print(f"Total unique teams across all seasons: {len(all_teams)}")
print(f"\nSample teams: {sorted(list(all_teams))[:10]}")

In [ ]:
# Check matches per season
matches_per_season = matches.groupby('season_label').size().sort_index()
print("Matches per season:")
print(matches_per_season.tail(10))

## Step 12: Save to Database

In [ ]:
# Save to SQLite database
matches.to_sql("matches", con=engine, if_exists="replace", index=False)

print(f"✅ Saved {len(matches)} matches to {DB_PATH}")
print(f"   Seasons: {matches['season_label'].nunique()}")
print(f"   Date range: {matches['date'].min().date()} to {matches['date'].max().date()}")

## 🎯 Checkpoint!

Excellent! We now have:
- ✅ 15+ seasons of Serie A data (2009-present)
- ✅ Clean match results in SQLite database
- ✅ Team names, scores, dates, seasons

**What we DON'T have yet:**
- Matchday numbers (need to calculate from dates)
- Standings by matchday

**Next steps:**
1. Add matchday calculation
2. Calculate cumulative standings
3. Build Streamlit dashboard

Ready to continue?